In [ ]:
!pip install torch==2.2.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.41.2
!pip install peft==0.11.1
!pip install bitsandbytes==0.43.1  # Consider upgrading to 0.45.0+ for improvements
!pip install datasets==2.19.1
!pip install accelerate==0.27.2
!pip install faiss-cpu
!pip uninstall numpy -y
!pip install numpy==1.26.4

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.2.2%2Bcu118-cp311-cp311-linux_x86_64.whl (819.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_cupti_cu11-11.8.87-py3-none-manylinux1_x86_64.whl (13.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
ls /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/       # adjust if your path is different

ls: cannot access '/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/

/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K


In [ ]:
!gunzip -k *.gz

In [ ]:
!ls -lh

total 675M
-rw------- 1 root root  17M Feb 22  2021 lbl.json
-rw------- 1 root root 3.3M Feb 22  2021 lbl.json.gz
-rw------- 1 root root 888K May 29 19:06 test_labels.txt
-rw------- 1 root root 938K May 29 19:06 train_labels.txt
-rw------- 1 root root 399M Feb 22  2021 trn.json
-rw------- 1 root root 184M Feb 22  2021 tst.json
-rw------- 1 root root  73M Feb 22  2021 tst.json.gz


In [ ]:
!tar -xzvf ./data/LF-AmazonTitles-131K.tar.gz -C ./data/

In [ ]:
!ls -l ./data/LF-AmazonTitles-131K/

In [ ]:
import json
import os

def convert_json_to_txt(json_file_path, text_output_path, label_output_path):
    """
    Reads a JSON-lines file and splits it into two .txt files:
    1. A file with the text content (from the 'title' field).
    2. A file with space-separated labels (from the 'target_ind' field).
    """
    print(f"Processing {json_file_path}...")
    if not os.path.exists(json_file_path):
        print(f"Error: File not found -> {json_file_path}")
        return

    try:
        with open(json_file_path, 'r', encoding='utf-8') as f_json, \
             open(text_output_path, 'w', encoding='utf-8') as f_text, \
             open(label_output_path, 'w', encoding='utf-8') as f_label:

            for line in f_json:
                # Each line is a JSON object
                data = json.loads(line)

                # 1. Get the title (or content if you prefer)
                # We add a newline character for the .txt file
                title = data.get("title", "")
                f_text.write(title + '\n')

                # 2. Get the labels, convert them to strings, and join with spaces
                labels = data.get("target_ind", [])
                label_str = ' '.join(map(str, labels))
                f_label.write(label_str + '\n')

        print(f"Successfully created: {text_output_path}")
        print(f"Successfully created: {label_output_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- Main execution ---
if __name__ == "__main__":
    # Assuming the script is in the same directory as trn.json, tst.json
    # Convert training data
    convert_json_to_txt(
        json_file_path='/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/tst.json',
        text_output_path='/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/train_texts.txt',
        label_output_path='/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/train_labels.txt'
    )

    print("-" * 20)

    # Convert test data
    convert_json_to_txt(
        json_file_path='/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/tst.json',
        text_output_path='/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/test_texts.txt',
        label_output_path='/content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/test_labels.txt'
    )

    print("\nConversion complete! Your dataset is now ready for the main script.")

Processing /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/tst.json...
Successfully created: /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/train_texts.txt
Successfully created: /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/train_labels.txt
--------------------
Processing /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/tst.json...
Successfully created: /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/test_texts.txt
Successfully created: /content/drive/MyDrive/LF-Amazon-131K.raw/LF-Amazon-131K/test_labels.txt

Conversion complete! Your dataset is now ready for the main script.


In [ ]:
# LoRA + 8‑bit Fine‑Tuning for Extreme Multi‑Label Retrieval (Redesigned for Robustness)
# Author: Shivaram Kumar (Systems Researcher)
# Original Time‑frame: Jan – Feb 2025
# Redesign: For error handling and Colab-friendliness
# Hardware target: single RTX 3060 Max‑Q (original), Google Colab T4 GPU
#
# -----------------------------------------------------------------------------
# This script covers the full reproducible pipeline:
#   • parameter‑efficient fine‑tuning of an MPNet retriever with LoRA (r=8)
#   • 8‑bit Adam optimiser via bitsandbytes
#   • recall@100 evaluation for extreme multi‑label retrieval (Amazon‑670K‑like)
#   • timing / throughput profiler
#   • automatic logging of cost–speed trade‑offs
#
# The code is organised as *one* Python file that can be executed in two modes:
#   1.  TRAIN   – fine‑tune and save the LoRA‑adapted 8‑bit model + tokenizer
#   2.  BENCH   – measure GPU memory, wall‑clock throughput and recall@100
#
# Example usage (after placing dataset in data/Amazon670K):
#   $ python lora_8bit_retrieval_robust.py train \
#       --dataset_path data/Amazon670K \
#       --output_dir exp/mpnet_lora8_8bit_robust
#   $ python lora_8bit_retrieval_robust.py bench \
#       --dataset_path data/Amazon670K \
#       --model_dir exp/mpnet_lora8_8bit_robust
# -----------------------------------------------------------------------------

#
# === FOR GOOGLE COLAB: Run this cell first ===
#
# !pip install torch==2.2.* torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.41.*
# !pip install peft==0.11.*
# !pip install bitsandbytes==0.43.*
# !pip install datasets==2.19.*
# !pip install faiss-cpu==1.8.* # or faiss-gpu if you prefer and have VRAM
# !pip install accelerate==0.28.*
# !pip install rich>=13
# !pip install sentence-transformers # For all-mpnet-base-v2, though AutoModel handles it
#
# ========================================
#

from __future__ import annotations

import argparse, os, sys, json, time, math, shutil, warnings
from contextlib import nullcontext
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional
import numpy as np # Added for type hinting

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Attempt to import, with guidance if missing
try:
    from datasets import load_dataset, DatasetDict # Not used in current AmazonXMC, but good for future
except ImportError:
    print("Hugging Face 'datasets' library not found. Please install it if you plan to use it.")

try:
    from transformers import (
        AutoTokenizer,
        AutoModel,
        TrainingArguments,
        Trainer,
        DataCollatorWithPadding,
    )
    from transformers.trainer_utils import get_last_checkpoint
except ImportError:
    sys.exit("Hugging Face 'transformers' library not found. Please install it: pip install transformers")

try:
    from peft import LoraConfig, get_peft_model, TaskType
except ImportError:
    sys.exit("Hugging Face 'peft' library not found. Please install it: pip install peft")

try:
    from bitsandbytes.optim import Adam8bit
except ImportError:
    sys.exit("'bitsandbytes' library not found. Please install it: pip install bitsandbytes")

try:
    import faiss
except ImportError:
    sys.exit("'faiss' library not found. Please install it: pip install faiss-cpu or faiss-gpu")

try:
    from rich.console import Console
    from rich.table import Table
    from rich.panel import Panel
    from rich.text import Text
except ImportError:
    # Fallback if rich is not installed
    class Console:
        def print(self, *args, **kwargs):
            print(*args)
    class Table:
        def __init__(self, *args, **kwargs): pass
        def add_column(self, *args, **kwargs): pass
        def add_row(self, *args, **kwargs): pass
    Panel = print
    Text = str


console = Console()

# -----------------------------
# Dataset Integrity and Setup
# -----------------------------
def ensure_dataset_integrity(dataset_root: str, required_files: List[str]) -> bool:
    """Checks if all required dataset files exist at the specified root."""
    if not os.path.isdir(dataset_root):
        console.print(f"[bold red]Error: Dataset directory not found: {dataset_root}[/]")
        return False

    missing_files = []
    for fname in required_files:
        fpath = os.path.join(dataset_root, fname)
        if not os.path.isfile(fpath):
            missing_files.append(fname)

    if missing_files:
        console.print(f"[bold red]Error: Missing dataset files in {dataset_root}:[/]")
        for mf in missing_files:
            console.print(f"  - {mf}")
        console.print("\n[bold yellow]Please ensure your dataset (e.g., Amazon-670K) is correctly formatted and placed.")
        console.print("The expected structure is:")
        console.print(f"  {dataset_root}/")
        console.print(f"    train_texts.txt")
        console.print(f"    train_labels.txt")
        console.print(f"    test_texts.txt")
        console.print(f"    test_labels.txt")
        console.print("\n[bold cyan]For Colab, you might need to:")
        console.print("  1. Upload your dataset (e.g., as a zip file) and unzip it.")
        console.print("     Example: Create a 'data/Amazon670K' folder in your Colab environment.")
        console.print("  2. Or, mount your Google Drive if the dataset is stored there:")
        console.print("     from google.colab import drive; drive.mount('/content/drive')")
        console.print("     Then adjust --dataset_path to, e.g., '/content/drive/MyDrive/data/Amazon670K'")
        return False
    return True

# -----------------------------
# Dataset helpers
# -----------------------------
class AmazonXMC(Dataset):
    """Thin wrapper exposing (text, labels) for Amazon‑670K‑style files.

    Expected directory layout (download links in README of original project):
      data/Amazon670K/
          train_texts.txt   – one product title per line
          train_labels.txt  – space‑separated integer labels per line
          test_texts.txt
          test_labels.txt

    Loading only the first *k* labels per sample is supported through
    `topk_labels` to keep memory bounded for quick experiments.
    """

    def __init__(
        self,
        root: str,
        split: str = "train",
        topk_labels: int | None = None,
        max_samples: int | None = None,
    ) -> None:
        assert split in {"train", "test"}, "Split must be 'train' or 'test'"
        self.text_fp = os.path.join(root, f"{split}_texts.txt")
        self.label_fp = os.path.join(root, f"{split}_labels.txt")
        self.texts: List[str] = []
        self.labels: List[List[int]] = []

        try:
            with open(self.text_fp, "r", encoding="utf‑8") as ft:
                self.texts = [l.rstrip("\n") for l in ft]
            with open(self.label_fp, "r", encoding="utf‑8") as fl:
                self.labels = [
                    [int(t) for t in l.rstrip("\n").split()[:topk_labels]]
                    for l in fl
                ]
        except FileNotFoundError as e:
            console.print(f"[bold red]Error: Dataset file not found: {e.filename}[/]")
            console.print(f"Please check the path: {root} and ensure '{split}_texts.txt' and '{split}_labels.txt' exist.")
            raise  # Re-raise the exception to stop execution if files are critical
        except Exception as e:
            console.print(f"[bold red]Error loading dataset files from {root}: {e}[/]")
            raise

        if len(self.texts) != len(self.labels):
            console.print(f"[bold red]Error: Number of texts ({len(self.texts)}) does not match number of label sets ({len(self.labels)}) in {split} set.[/]")
            raise ValueError("Mismatch in number of text samples and label samples.")

        if max_samples:
            if max_samples > len(self.texts):
                console.print(f"[yellow]Warning: max_samples ({max_samples}) is greater than available samples ({len(self.texts)}). Using all available samples.[/]")
            self.texts = self.texts[: max_samples]
            self.labels = self.labels[: max_samples]

        if not self.texts:
            console.print(f"[bold yellow]Warning: Loaded 0 samples for split '{split}' from {root}. This might lead to errors downstream.[/]")


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if not (0 <= idx < len(self.texts)):
            raise IndexError(f"Index {idx} out of bounds for dataset with length {len(self.texts)}")
        return {"text": self.texts[idx], "labels": self.labels[idx]}


# -----------------------------
# Model + LoRA helpers
# -----------------------------

def load_backbone(model_name: str, load_in_8bit: bool = True):
    console.print(f"[bold cyan]Loading {model_name} (8‑bit={load_in_8bit})...[/]")

    # MPNet models have compatibility issues with quantization
    is_mpnet = "mpnet" in model_name.lower()
    use_quantization = load_in_8bit and not is_mpnet

    if is_mpnet and load_in_8bit:
        console.print("[yellow]Warning: MPNet models have issues with 8-bit quantization. Loading with fp16 instead.[/]")

    try:
        if use_quantization:
            from transformers import BitsAndBytesConfig
            quantization_config = BitsAndBytesConfig(load_in_8bit=True)

            model = AutoModel.from_pretrained(
                model_name,
                quantization_config=quantization_config,
                device_map="auto",  # Safe for non-MPNet models
                trust_remote_code=True,
            )
        else:
            # For MPNet: use fp16 instead of quantization
            model = AutoModel.from_pretrained(
                model_name,
                torch_dtype=torch.float16,  # Half precision for memory efficiency
                trust_remote_code=True,
            )
            # Manually move to GPU for non-quantized models
            if torch.cuda.is_available():
                model = model.cuda()

        #console.print("[bold yellow]Inspecting model named_modules():[/]")
        #for name, module in model.named_modules():
        #    console.print(f"  - {name}")
        #console.print("[/]")

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.truncation_side = "right"

        if tokenizer.pad_token is None:
            console.print("[yellow]Warning: Tokenizer does not have a PAD token. Setting to EOS token.[/]")
            tokenizer.pad_token = tokenizer.eos_token

        console.print(f"[green]✓ Model {model_name} loaded successfully (quantized: {use_quantization}).[/]")
        return model, tokenizer
    except Exception as e:
        console.print(f"[bold red]Error loading model or tokenizer '{model_name}': {e}[/]")
        raise



def inject_lora(model, r: int = 8, alpha: int = 16, dropout: float = 0.1):
    """Wrap key projection modules with LoRA adapters."""
    # Target modules depend on the model architecture. For MPNet:
    #target_modules = ["q_proj", "v_proj"] # Correct for MPNet-like models from sentence-transformers
    target_modules = ["q", "v"]
    # For other BERT-like models, it might be ["query", "value"]
    # It's good to inspect model.named_modules() to confirm target_modules for a new architecture.

    lora_cfg = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION, # Using this as we are extracting embeddings
        r=r,
        lora_alpha=alpha,
        lora_dropout=dropout,
        bias="none", # Recommended for LoRA
        target_modules=target_modules,
    )
    try:
        lora_model = get_peft_model(model, lora_cfg)
        # Ensure trainable LoRA parameters are float32 when using AMP (fp16=True)
        # This is crucial if the base model was loaded in fp16.
        for param in lora_model.parameters():
            if param.requires_grad:
                param.data = param.data.float()
        console.print(f"[green]✓ LoRA adapters (r={r}, alpha={alpha}) injected.[/]")
        lora_model.print_trainable_parameters()
        return lora_model
    except Exception as e:
        console.print(f"[bold red]Error injecting LoRA adapters: {e}[/]")
        console.print("[bold yellow]Potential issues:[/]")
        console.print(f"  - `target_modules` ({target_modules}) might be incorrect for the loaded model architecture.")
        console.print("  - Incompatibility with the 'peft' library version.")
        raise

# -----------------------------
# Trainer plumbing
# -----------------------------

def collate_fn(tokenizer, max_length=128): # Made max_length a parameter
    # DataCollatorWithPadding handles padding to the longest sequence in the batch
    # It's generally preferred over tokenizer(..., padding='max_length') unless all sequences must be padded to max_length
    # For dynamic padding:
    # data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest')
    # However, the original code tokenizes manually, which is fine.

    def _fn(batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        texts = [b["text"] for b in batch]
        labels = [b["labels"] for b in batch] # List of lists of integers

        try:
            # Ensure max_length is reasonable for the model
            # Most sentence transformers like MPNet have a max sequence length of 128-512
            # Using a fixed max_length here for consistent tensor shapes if desired,
            # but padding='longest' (default for DataCollatorWithPadding) is often more efficient.
            tokenised = tokenizer(
                texts,
                padding="longest", # Pad to longest in batch
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            # The original had `labels` as a list of lists, which is fine for custom eval
            # but if Trainer's compute_metrics or loss needed it as tensors, it would need adjustment.
            # Here, `compute_loss` handles the embeddings directly, and eval is custom.
            tokenised["labels"] = labels
            return tokenised
        except Exception as e:
            console.print(f"[bold red]Error during tokenization or collation: {e}[/]")
            # Log a snippet of the problematic text if possible and safe
            if texts:
                console.print(f"Problematic text sample (first 50 chars): '{texts[0][:50]}...'")
            raise
    return _fn


def compute_loss(model, batch_inputs, device): # Added device parameter
    """Contrastive / multi‑label retrieval loss (pairwise NT‑Xent light)."""
    if device is None:
      device = next(model.parameters()).device

    # Prepare inputs for the model, ensuring they are on the correct device
    model_inputs = {k: v.to(device) for k, v in batch_inputs.items() if k != "labels" and hasattr(v, 'to')}

    try:
        outputs = model(**model_inputs)
        # CLS token embedding is typically at index 0 of the first token's hidden state
        emb = outputs.last_hidden_state[:, 0, :]  # (Batch, HiddenDim)
        emb = F.normalize(emb, p=2, dim=-1) # L2 normalization

        # Cosine similarity matrix
        sim_matrix = emb @ emb.T  # (Batch, Batch)

        # NT-Xent Loss: treats each sample as its own class (positive example)
        # and all other samples in the batch as negative examples.
        # Labels for cross_entropy are just 0, 1, 2, ..., B-1
        labels = torch.arange(emb.size(0), device=device)

        # Temperature scaling (optional, but common in contrastive learning)
        # temperature = 0.07 # Example value
        # loss = F.cross_entropy(sim_matrix / temperature, labels)
        loss = F.cross_entropy(sim_matrix, labels)
        return loss
    except Exception as e:
        console.print(f"[bold red]Error during loss computation: {e}[/]")
        # Potentially log shapes of tensors involved if a shape mismatch error occurs
        console.print(f"Input keys: {model_inputs.keys()}")
        if 'last_hidden_state' in outputs:
             console.print(f"Shape of last_hidden_state: {outputs.last_hidden_state.shape}")
        raise

class RetrievalTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        device = next(model.parameters()).device
        # `inputs` already contains tensors on the correct device from DataLoader
        loss = compute_loss(model, inputs, device) # Pass model.device
        # `return_outputs=True` expects (loss, outputs_dict)
        # Here, outputs_dict is not strictly necessary if we only care about loss for training step
        return (loss, {"loss": loss}) if return_outputs else loss

    # prediction_step is used for evaluation if `compute_metrics` is defined
    # or if `predict()` is called.
    def prediction_step(
        self,
        model: torch.nn.Module,
        inputs: Dict[str, Any],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:
        device = next(model.parameters()).device
        model_inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "labels" and hasattr(v, 'to')}
        raw_labels = inputs["labels"] # Keep raw labels (list of lists of int)

        with torch.no_grad():
            try:
                outputs = model(**model_inputs)
                emb = outputs.last_hidden_state[:, 0, :]
                emb = F.normalize(emb, p=2, dim=-1)
            except Exception as e:
                console.print(f"[bold red]Error during prediction_step (model inference): {e}[/]")
                # Return None for embeddings if error occurs, Trainer might handle this
                return (None, None, None)
        placeholder_labels = torch.arange(emb.size(0), device=emb.device)
        # No loss computed here by default unless `prediction_loss_only` is False and `compute_loss` is adapted
        # For current custom bench, we only need embeddings and labels.
        # The trainer expects (optional_loss, logits_or_embeddings, labels_for_metrics)
        # We return None for loss, our embeddings, and the raw labels.
        #return (None, emb.cpu(), raw_labels)
        return (None, emb.cpu(), placeholder_labels)




# -----------------------------
# recall@K evaluation (Corrected Logic)
# -----------------------------

def build_faiss_index(embs: np.ndarray, dim: int) -> Optional[faiss.Index]:
    try:
        # Using IndexFlatIP because embeddings are normalized (IP = Cosine Similarity)
        index = faiss.IndexFlatIP(dim)
        index.add(embs)
        console.print(f"[green]✓ FAISS index built with {embs.shape[0]} embeddings of dim {dim}.[/]")
        return index
    except Exception as e:
        console.print(f"[bold red]Error building FAISS index: {e}[/]")
        console.print("[bold yellow]Ensure FAISS is correctly installed and embeddings are valid (numpy array, correct dtype).[/]")
        return None


def recall_at_k(
    query_embeddings: np.ndarray,
    query_true_labels: List[List[int]], # Labels for each query [[1,2], [3], ...]
    corpus_item_embeddings: np.ndarray, # Embeddings of all items in the retrieval corpus
    corpus_item_true_labels: List[List[int]], # Labels for each item in the corpus
    faiss_index: faiss.Index, # Pre-built FAISS index on corpus_item_embeddings
    k_val: int = 100,
) -> float:
    """
    Calculates recall@k for multi-label retrieval.
    A query is considered a "hit" if any of its true labels are found
    among the true labels of any of the top-k retrieved corpus items.
    """
    if query_embeddings.shape[0] != len(query_true_labels):
        console.print(f"[bold red]Mismatch in query embeddings ({query_embeddings.shape[0]}) and query labels ({len(query_true_labels)}) count.[/]")
        return 0.0
    if faiss_index.ntotal != len(corpus_item_true_labels):
        console.print(f"[bold red]Mismatch in FAISS index size ({faiss_index.ntotal}) and corpus labels count ({len(corpus_item_true_labels)}).[/]")
        return 0.0

    try:
        # D: distances (similarity scores), I: indices of retrieved corpus items
        D, I = faiss_index.search(query_embeddings, k_val)
    except Exception as e:
        console.print(f"[bold red]Error during FAISS search: {e}[/]")
        return 0.0

    num_queries = query_embeddings.shape[0]
    hits = 0

    for i in range(num_queries):
        current_query_true_labels_set = set(query_true_labels[i])
        if not current_query_true_labels_set: # Skip if query has no true labels
            continue

        retrieved_corpus_indices = I[i] # Indices of top-k items from corpus

        found_match_for_this_query = False
        for corpus_idx in retrieved_corpus_indices:
            if corpus_idx == -1: # FAISS can return -1 if fewer than k items are found
                continue

            # Get the true labels of the retrieved corpus item
            retrieved_corpus_item_labels_set = set(corpus_item_true_labels[corpus_idx])

            # Check for intersection
            if not current_query_true_labels_set.isdisjoint(retrieved_corpus_item_labels_set):
                found_match_for_this_query = True
                break # Move to next query once a match is found

        if found_match_for_this_query:
            hits += 1

    if num_queries == 0:
        return 0.0

    recall = (hits / num_queries) * 100.0
    return recall


# -----------------------------
# Training routine
# -----------------------------

def train(args):
    console.print(Panel("[bold green]Initializing Training Process[/]", expand=False))


     # --- BEGIN ADDED CHECK TO SKIP COMPLETED TRAINING ---
    adapter_model_path_safetensors = os.path.join(args.output_dir, "adapter_model.safetensors")
    adapter_model_path_bin = os.path.join(args.output_dir, "adapter_model.bin") # Older format
    adapter_config_path = os.path.join(args.output_dir, "adapter_config.json")
    tokenizer_config_path = os.path.join(args.output_dir, "tokenizer_config.json") # Also check tokenizer

    final_adapter_exists = os.path.exists(adapter_model_path_safetensors) or os.path.exists(adapter_model_path_bin)

    if final_adapter_exists and os.path.exists(adapter_config_path) and os.path.exists(tokenizer_config_path):
        console.print(f"[bold yellow]Final LoRA adapter model and tokenizer already found in {args.output_dir}.[/]")
        console.print(f"[yellow]Skipping fine-tuning. To re-train, clear or use a different --output_dir.[/]")

        # Optionally, verify if number of epochs matches or if a 'completed.flag' file exists
        # For now, existence of key output files is a good heuristic.
        return


    required_files = ["train_texts.txt", "train_labels.txt", "test_texts.txt", "test_labels.txt"]
    if not ensure_dataset_integrity(args.dataset_path, required_files):
        return

    try:
        # 1. Load backbone + LoRA
        model, tokenizer = load_backbone(args.model_name, load_in_8bit=True)
        # Make sure PAD token is set if tokenizer doesn't have one (e.g., for Llama if used)
        if tokenizer.pad_token_id is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = tokenizer.eos_token_id

        model = inject_lora(model, r=args.lora_rank, alpha=args.lora_alpha) # Added lora_alpha

        # 2. Prepare dataset
        console.print(f"[cyan]Loading datasets from {args.dataset_path}...[/]")
        train_ds = AmazonXMC(args.dataset_path, "train", max_samples=args.train_samples, topk_labels=args.topk_labels_train)
        val_ds = AmazonXMC(args.dataset_path, "test", max_samples=args.val_samples, topk_labels=args.topk_labels_val)

        if len(train_ds) == 0:
            console.print("[bold red]Training dataset is empty. Aborting training.[/]")
            return
        if len(val_ds) == 0:
            console.print("[bold yellow]Warning: Validation dataset is empty. Evaluation will be skipped or may fail.[/]")

        # The collate_fn will use tokenizer.model_max_length if args.max_seq_length is not set.
        # It's good practice to set it explicitly or ensure the tokenizer has a sensible default.
        effective_max_length = args.max_seq_length if args.max_seq_length else tokenizer.model_max_length
        if not effective_max_length or effective_max_length > 2048: # Safety for very large/missing max_length
            console.print(f"[yellow]Warning: tokenizer.model_max_length is {tokenizer.model_max_length}. Clamping to 512 for collation.[/yellow]")
            effective_max_length = 512


        # 3. TrainingArguments w/ 8‑bit Adam
        # Ensure output_dir exists
        os.makedirs(args.output_dir, exist_ok=True)

        targs = TrainingArguments(
            output_dir=args.output_dir,
            num_train_epochs=args.epochs,
            per_device_train_batch_size=args.batch_size,
            per_device_eval_batch_size=args.eval_batch_size,
            gradient_accumulation_steps=args.grad_accum,
            fp16=True, # Mixed precision training
            evaluation_strategy="epoch" if len(val_ds) > 0 else "no",
            save_strategy="epoch", # Save model at the end of each epoch
            logging_steps=args.logging_steps, # Log every N steps
            save_total_limit=args.save_total_limit, # Only keep the last N checkpoints
            learning_rate=args.lr,
            weight_decay=args.weight_decay,
            report_to=args.report_to.split(',') if args.report_to else ["none"], # e.g. "tensorboard,wandb"
            dataloader_num_workers=args.num_workers, # For faster data loading
            remove_unused_columns=False, # Important because we keep "labels"
            load_best_model_at_end=False if len(val_ds) > 0 else False, # Load best model based on eval
            metric_for_best_model=None if len(val_ds) > 0 else None, # Default to loss if no custom metric
            greater_is_better=None if len(val_ds) > 0 else None,
        )

        # Custom optimizer: Adam8bit
        # The Trainer will use model.parameters() by default.
        # For LoRA, we only want to optimize trainable parameters.
        optimizer = Adam8bit(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr)
        # lr_scheduler is typically handled by Trainer, but can be passed too. (optimizer, scheduler)

        trainer = RetrievalTrainer(
            model=model,
            args=targs,
            train_dataset=train_ds,
            eval_dataset=val_ds if len(val_ds) > 0 else None,
            data_collator=collate_fn(tokenizer, max_length=effective_max_length),
            optimizers=(optimizer, None), # Pass the custom optimizer
        )
        # If you had custom compute_metrics for evaluation:
        # trainer.compute_metrics = compute_metrics_function

        # 4. Train
        console.print("[bold green]Starting fine‑tuning...[/]")
        # Check for last checkpoint
        last_checkpoint_dir = get_last_checkpoint(args.output_dir)
        valid_checkpoint_to_resume = None
        if last_checkpoint_dir:
            # Check if the found checkpoint is complete by looking for trainer_state.json
            trainer_state_path = os.path.join(last_checkpoint_dir, "trainer_state.json")
            adapter_model_path_sft = os.path.join(last_checkpoint_dir, "adapter_model.safetensors")
            adapter_model_path_bin = os.path.join(last_checkpoint_dir, "adapter_model.bin")
            pytorch_model_path = os.path.join(last_checkpoint_dir, "pytorch_model.bin") # For full model checkpoints

            # A checkpoint is considered valid if trainer_state.json and some model weights exist
            has_trainer_state = os.path.exists(trainer_state_path)
            has_adapter_model = os.path.exists(adapter_model_path_sft) or os.path.exists(adapter_model_path_bin)
            has_pytorch_model = os.path.exists(pytorch_model_path) # Might be saved by older trainer or non-PEFT

            if has_trainer_state and (has_adapter_model or has_pytorch_model):
                valid_checkpoint_to_resume = last_checkpoint_dir
                console.print(f"[yellow]Resuming training from complete checkpoint: {valid_checkpoint_to_resume}[/yellow]")
            else:
                console.print(f"[bold orange_red1]Warning: Last checkpoint directory '{last_checkpoint_dir}' found but appears incomplete (missing trainer_state.json or model files).[/]")
                # Try to find the second to last checkpoint if save_total_limit allows
                # This is more complex, for now, we'll just warn and potentially start fresh or let user handle.
                # A simple approach is to just not resume if the last one is broken.
                # For a more robust solution, you might list all checkpoint-* dirs, sort them,
                # and iterate backwards checking for completeness.
                console.print(f"[cyan]Attempting to find an older complete checkpoint or starting fresh.[/]")
                # As a basic recovery, let's try to see if Trainer can handle a potentially problematic path
                # or if it's better to pass None and force fresh start if last is broken.
                # For now, if it's incomplete, we won't resume from it with this explicit check.
                # last_checkpoint_dir = None # Effectively start fresh if most recent is broken.
                # OR, you could implement logic to find the *next oldest* valid one.
                # For this fix, let's just inform the user and proceed with `None` if incomplete.
                if not has_trainer_state:
                    console.print(f"  - Missing: {trainer_state_path}")
                if not (has_adapter_model or has_pytorch_model):
                    console.print(f"  - Missing: Model files (e.g., adapter_model.safetensors or pytorch_model.bin)")

                # Attempt to find the next valid checkpoint by looking at parent directory's checkpoints
                # This is a simplified version. Transformers has internal logic too.
                all_checkpoints = [os.path.join(args.output_dir, d) for d in os.listdir(args.output_dir) if d.startswith("checkpoint-")]
                all_checkpoints.sort(key=lambda x: int(x.split("-")[-1]))

                for chkpt_path in reversed(all_checkpoints):
                    if chkpt_path == last_checkpoint_dir: # Skip the one we know is bad
                        continue
                    ts_path = os.path.join(chkpt_path, "trainer_state.json")
                    am_path_sft = os.path.join(chkpt_path, "adapter_model.safetensors")
                    am_path_bin = os.path.join(chkpt_path, "adapter_model.bin")
                    pm_path = os.path.join(chkpt_path, "pytorch_model.bin")
                    if os.path.exists(ts_path) and (os.path.exists(am_path_sft) or os.path.exists(am_path_bin) or os.path.exists(pm_path)):
                        valid_checkpoint_to_resume = chkpt_path
                        console.print(f"[yellow]Found older complete checkpoint to resume from: {valid_checkpoint_to_resume}[/yellow]")
                        break
                if not valid_checkpoint_to_resume:
                     console.print(f"[cyan]No complete checkpoint found in {args.output_dir}. Starting fresh training.[/]")
        else:
            console.print(f"[cyan]No checkpoint found in {args.output_dir}. Starting fresh training.[/]")

        train_result = trainer.train(resume_from_checkpoint=last_checkpoint_dir)
        console.print(Panel(f"[bold green]✓ Training finished. Results: {train_result.metrics}[/]", expand=False))

        # 5. Save final model, tokenizer, and training state
        trainer.save_model(args.output_dir) # Saves LoRA adapter and config
        # To save the full model (if needed, though LoRA is usually separate):
        # model.save_pretrained(os.path.join(args.output_dir, "full_model"))
        tokenizer.save_pretrained(args.output_dir)
        trainer.save_state() # Saves optimizer state, RNG state, etc.

        console.print(f"[bold green]✓ Model, tokenizer, and training state saved to {args.output_dir}[/]")

    except FileNotFoundError as e:
        console.print(f"[bold red]Training Error: A required file was not found: {e}[/]")
    except ValueError as e:
        console.print(f"[bold red]Training Error: A value was incorrect: {e}[/]")
    except RuntimeError as e: # Catch CUDA errors, etc.
        console.print(f"[bold red]Training Runtime Error: {e}[/]")
        console.print("[bold yellow]This could be a CUDA OOM error. Try reducing batch size or sequence length.[/]")
    except Exception as e:
        console.print(f"[bold red]An unexpected error occurred during training: {e}[/]")
        import traceback
        traceback.print_exc()


# -----------------------------
# Benchmark routine
# -----------------------------
def bench(args):
    device = next(model.parameters()).device
    console.print(Panel("[bold blue]Initializing Benchmarking Process[/]", expand=False))

    # Ensure model directory exists
    if not os.path.isdir(args.model_dir):
        console.print(f"[bold red]Error: Model directory not found: {args.model_dir}[/]")
        console.print("Please ensure you have a trained model saved (e.g., after running the 'train' command).")
        return

    required_files = ["train_texts.txt", "train_labels.txt", "test_texts.txt", "test_labels.txt"] # Corpus needs train files
    if not ensure_dataset_integrity(args.dataset_path, required_files):
        return

    # In the bench function, replace the model loading section with:
    if load_in_8bit:
        from transformers import BitsAndBytesConfig
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)

        model = AutoModel.from_pretrained(
            base_model_name,
            quantization_config=quantization_config,
            #device_map="auto",
            trust_remote_code=True,
        )
    else:
        model = AutoModel.from_pretrained(
            base_model_name,
            #device_map="auto",
            trust_remote_code=True,
        )

    # Then apply LoRA adapters
    model = get_peft_model(model, LoraConfig.from_pretrained(args.model_dir))


    try:
        # 1. Load fine-tuned LoRA model (PEFT will handle loading adapters)
        # When loading a PEFT model, you typically load the base model first, then apply adapters.
        # However, if `save_pretrained` was used on a PeftModel, it saves adapter config.
        # Let's assume the `model_dir` contains the LoRA adapter and the original base model name is known or saved.
        # For simplicity, we reload the base model and then apply LoRA weights from `model_dir`.

        # First, determine the base model name (this might be saved in adapter_config.json or known)
        # For this script, we assume the same base model as training.
        # A more robust way would be to save base_model_name_or_path in PeftConfig.
        base_model_name = args.base_model_name_for_bench # Add this as an arg or infer

        model, tokenizer = load_backbone(base_model_name, load_in_8bit=True)
        model = get_peft_model(model, LoraConfig.from_pretrained(args.model_dir)) # Load LoRA adapters

        # Alternatively, if you saved the full model with adapters merged:
        # model = AutoModel.from_pretrained(args.model_dir, load_in_8bit=True, device_map="auto")
        # tokenizer = AutoTokenizer.from_pretrained(args.model_dir)

        model.eval() # Set model to evaluation mode
        console.print(f"[green]✓ Fine-tuned LoRA model and tokenizer loaded from {args.model_dir}.[/]")

        effective_max_length = args.max_seq_length if args.max_seq_length else tokenizer.model_max_length
        if not effective_max_length or effective_max_length > 2048:
            effective_max_length = 512


        # 2. Prepare datasets for corpus and queries
        console.print("[cyan]Loading corpus (from training set) and query (from test set) data...[/]")
        # Corpus for FAISS index (typically from training data)
        corpus_dataset = AmazonXMC(args.dataset_path, "train", max_samples=args.index_samples, topk_labels=args.topk_labels_val) # Use val topk for consistency
        # Query dataset (typically from test data)
        query_dataset = AmazonXMC(args.dataset_path, "test", max_samples=args.val_samples, topk_labels=args.topk_labels_val)

        if len(corpus_dataset) == 0 or len(query_dataset) == 0:
            console.print("[bold red]Corpus or query dataset is empty. Aborting benchmark.[/]")
            return

        corpus_loader = DataLoader(corpus_dataset, batch_size=args.eval_batch_size, shuffle=False, collate_fn=collate_fn(tokenizer, max_length=effective_max_length))
        query_loader = DataLoader(query_dataset, batch_size=args.eval_batch_size, shuffle=False, collate_fn=collate_fn(tokenizer, max_length=effective_max_length))

        dim = model.config.hidden_size # Embedding dimension
        device = next(model.parameters()).device

        # Helper to encode datasets
        def encode_dataset(loader: DataLoader, model_to_encode_with: torch.nn.Module) -> np.ndarray:
            all_embeddings = []
            model_to_encode_with.eval() # Ensure model is in eval mode
            with torch.no_grad():
                for batch in loader:
                    model_inputs = {k: v.to(device) for k, v in batch.items() if k != "labels" and hasattr(v, 'to')}
                    outputs = model_to_encode_with(**model_inputs)
                    emb = outputs.last_hidden_state[:, 0, :]
                    emb = F.normalize(emb, p=2, dim=-1)
                    all_embeddings.append(emb.cpu().numpy())
            return np.concatenate(all_embeddings, axis=0)

        # 3. Encode corpus and build FAISS index
        console.print("[cyan]Encoding corpus items...[/]")
        t_start_corpus_encode = time.perf_counter()
        corpus_embeddings_np = encode_dataset(corpus_loader, model)
        corpus_encode_time = time.perf_counter() - t_start_corpus_encode
        console.print(f"  Corpus encoding took: {corpus_encode_time:.2f}s for {len(corpus_embeddings_np)} items.")

        t_start_faiss_build = time.perf_counter()
        faiss_index = build_faiss_index(corpus_embeddings_np, dim)
        if faiss_index is None: return # Error handled in build_faiss_index
        faiss_build_time = time.perf_counter() - t_start_faiss_build
        console.print(f"  FAISS index build took: {faiss_build_time:.2f}s.")

        build_time_total = corpus_encode_time + faiss_build_time

        # 4. Encode queries and perform search
        console.print("[cyan]Encoding query items...[/]")
        t_start_query_encode = time.perf_counter()
        query_embeddings_np = encode_dataset(query_loader, model)
        query_encode_time = time.perf_counter() - t_start_query_encode
        console.print(f"  Query encoding took: {query_encode_time:.2f}s for {len(query_embeddings_np)} items.")

        # Prepare labels for recall calculation
        corpus_true_labels_list = corpus_dataset.labels # List of lists from AmazonXMC
        query_true_labels_list = query_dataset.labels   # List of lists from AmazonXMC

        console.print(f"[cyan]Performing search and calculating recall@{args.recall_k_val}...[/]")
        t_start_search = time.perf_counter()
        recall = recall_at_k(
            query_embeddings_np,
            query_true_labels_list,
            corpus_embeddings_np, # Pass corpus embeddings (not strictly needed if index is passed)
            corpus_true_labels_list,
            faiss_index,
            k_val=args.recall_k_val
        )
        search_time = time.perf_counter() - t_start_search
        console.print(f"  FAISS search and recall calculation took: {search_time:.2f}s.")

        # Throughput calculation
        total_query_processing_time = query_encode_time + search_time
        qps = len(query_dataset) / total_query_processing_time if total_query_processing_time > 0 else 0

        # 5. Log results
        # Ensure model_dir for saving results exists (it should if model was loaded)
        os.makedirs(args.model_dir, exist_ok=True)
        summary_save_path = os.path.join(args.model_dir, f"bench_summary_k{args.recall_k_val}.json")

        # GPU Memory
        peak_mem_gb = 0.0
        if torch.cuda.is_available():
            peak_mem_gb = torch.cuda.max_memory_allocated() / 1e9
            torch.cuda.reset_peak_memory_stats() # Reset for next potential measurement

        summary = {
            "timestamp": datetime.now().isoformat(timespec="seconds"),
            "model_dir": args.model_dir,
            "base_model_name": base_model_name,
            "dataset_path": args.dataset_path,
            "index_samples": len(corpus_dataset),
            "query_samples": len(query_dataset),
            f"recall@{args.recall_k_val}": recall,
            "throughput_qps (queries_encoded_and_searched/sec)": qps,
            "corpus_encode_time_s": corpus_encode_time,
            "faiss_index_build_time_s": faiss_build_time,
            "total_index_build_time_s": build_time_total,
            "query_encode_time_s": query_encode_time,
            "search_and_recall_calc_time_s": search_time,
            "gpu_peak_mem_gb": peak_mem_gb,
        }

        with open(summary_save_path, "w") as fp:
            json.dump(summary, fp, indent=4)

        console.print(Panel(f"[bold green]✓ Benchmarking Complete. Recall@{args.recall_k_val}: {recall:.2f}%[/]", expand=False))

        # Rich table output
        table = Table(title=f"Benchmark Summary (Recall@{args.recall_k_val}) - {args.model_dir}")
        table.add_column("Metric", style="cyan")
        table.add_column("Value", style="magenta")
        for key, value in summary.items():
            if isinstance(value, float):
                table.add_row(key, f"{value:.4f}")
            else:
                table.add_row(key, str(value))
        console.print(table)
        console.print(f"[bold green]Summary saved to → {summary_save_path}[/]")

    except FileNotFoundError as e:
        console.print(f"[bold red]Benchmarking Error: A required file was not found: {e}[/]")
    except ValueError as e:
        console.print(f"[bold red]Benchmarking Error: A value was incorrect: {e}[/]")
    except RuntimeError as e: # Catch CUDA errors, etc.
        console.print(f"[bold red]Benchmarking Runtime Error: {e}[/]")
        console.print("[bold yellow]This could be a CUDA OOM error. Try reducing batch size or number of samples.[/]")
    except Exception as e:
        console.print(f"[bold red]An unexpected error occurred during benchmarking: {e}[/]")
        import traceback
        traceback.print_exc()

# -----------------------------
# CLI
# -----------------------------

def parse_args():
    p = argparse.ArgumentParser(
        description="LoRA + 8‑bit Retriever fine‑tuning & benchmarking (Robust Version)",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    sub = p.add_subparsers(dest="command", required=True, help="Specify 'train' or 'bench'")

    # Common args for model and data
    p_common = argparse.ArgumentParser(add_help=False)
    p_common.add_argument("--dataset_path", type=str, required=True, help="Path to the root of the dataset (e.g., data/Amazon670K)")
    p_common.add_argument("--model_name", type=str, default="sentence-transformers/all-mpnet-base-v2", help="Base model name from Hugging Face Hub")
    p_common.add_argument("--max_seq_length", type=int, default=128, help="Maximum sequence length for tokenizer")
    p_common.add_argument("--topk_labels_train", type=int, default=None, help="Use only top K labels per sample for training (None for all)")
    p_common.add_argument("--topk_labels_val", type=int, default=None, help="Use only top K labels per sample for validation/testing (None for all)")


    # TRAIN arguments
    p_train = sub.add_parser("train", parents=[p_common], help="Fine-tune a model", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_train.add_argument("--output_dir", type=str, required=True, help="Directory to save trained model checkpoints and logs")
    p_train.add_argument("--epochs", type=int, default=1, help="Number of training epochs")
    p_train.add_argument("--batch_size", type=int, default=12, help="Training batch size per device")
    p_train.add_argument("--eval_batch_size", type=int, default=32, help="Evaluation batch size per device")
    p_train.add_argument("--grad_accum", type=int, default=2, help="Gradient accumulation steps")
    p_train.add_argument("--lr", type=float, default=2e-5, help="Learning rate")
    p_train.add_argument("--weight_decay", type=float, default=0.01, help="Weight decay for optimizer")
    p_train.add_argument("--lora_rank", type=int, default=8, help="LoRA rank (r)")
    p_train.add_argument("--lora_alpha", type=int, default=16, help="LoRA alpha")
    p_train.add_argument("--train_samples", type=int, default=500_000, help="Max training samples to use (None for all)")
    p_train.add_argument("--val_samples", type=int, default=5_000, help="Max validation samples to use (None for all)")
    p_train.add_argument("--logging_steps", type=int, default=100, help="Log training metrics every N steps")
    p_train.add_argument("--save_total_limit", type=int, default=1, help="Max number of checkpoints to keep")
    p_train.add_argument("--num_workers", type=int, default=2, help="Number of dataloader workers")
    p_train.add_argument("--report_to", type=str, default="none", help="Reporting integrations (e.g. 'tensorboard', 'wandb', 'none')")


    # BENCH arguments
    p_bench = sub.add_parser("bench", parents=[p_common], help="Benchmark a trained model", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_bench.add_argument("--model_dir", type=str, required=True, help="Directory of the fine-tuned LoRA model (adapters)")
    # Need to specify base model for bench too, as LoRA adapters are applied on top.
    p_bench.add_argument("--base_model_name_for_bench", type=str, default="sentence-transformers/all-mpnet-base-v2", help="Base model name that was used for fine-tuning the adapters in --model_dir")
    p_bench.add_argument("--eval_batch_size", type=int, default=64, help="Batch size for encoding corpus and queries")
    p_bench.add_argument("--val_samples", type=int, default=10_000, help="Max query samples for benchmarking (from test set)")
    p_bench.add_argument("--index_samples", type=int, default=100_000, help="Max corpus samples for FAISS index (from train set)")
    p_bench.add_argument("--recall_k_val", type=int, default=100, help="Value of K for recall@K calculation")

    # Override model_name for bench as it's implicitly defined by base_model_name_for_bench and model_dir
    # This avoids conflict if --model_name is also passed with bench command by mistake.
    p_bench.set_defaults(model_name=None)

    return p.parse_args()


if __name__ == "__main__":
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Clear cache at the very beginning


    import sys

    sys.argv = [
        'my_script_name.py', # Placeholder for script name
        'train',
        '--dataset_path', '/content/drive/MyDrive/LORA',
        '--output_dir', '/content/exp/mpnet_lora8_8bit_robust/',
        '--epochs', '1',
        '--model_name', 'sentence-transformers/all-mpnet-base-v2',
        '--lora_rank', '8',
        '--lora_alpha', '16'
        # ... add other necessary arguments for train or bench
    ]

    args = parse_args()

    console.print(Panel(f"[bold bright_magenta]Executing Command: {args.command}[/]", expand=False, border_style="bright_magenta"))
    console.print(f"Arguments: {vars(args)}")

    if args.command == "train":
        # For training, args.model_name is the base model to start from
        if not args.model_name: # From p_common, should be set unless explicitly None
             console.print("[bold red]Error: --model_name must be specified for training.[/]")
             sys.exit(1)
        train(args)
    elif args.command == "bench":
        # For benchmarking, args.base_model_name_for_bench is the original base
        # and args.model_dir contains the LoRA adapters.
        if not args.base_model_name_for_bench:
            console.print("[bold red]Error: --base_model_name_for_bench must be specified for benchmarking to load adapters correctly.[/]")
            sys.exit(1)
        bench(args)
    else:
        console.print(f"[bold red]Unknown command: {args.command}[/]")
        sys.exit(1)

╭──────────────────────────╮
│ Executing Command: train │
╰──────────────────────────╯

Arguments: {'command': 'train', 'dataset_path': '/content/drive/MyDrive/LORA', 'model_name': 
'sentence-transformers/all-mpnet-base-v2', 'max_seq_length': 128, 'topk_labels_train': None, 'topk_labels_val': 
None, 'output_dir': '/content/exp/mpnet_lora8_8bit_robust/', 'epochs': 1, 'batch_size': 12, 'eval_batch_size': 32, 
'grad_accum': 2, 'lr': 2e-05, 'weight_decay': 0.01, 'lora_rank': 8, 'lora_alpha': 16, 'train_samples': 500000, 
'val_samples': 5000, 'logging_steps': 100, 'save_total_limit': 1, 'num_workers': 2, 'report_to': 'none'}

╭───────────────────────────────╮
│ Initializing Training Process │
╰───────────────────────────────╯

Final LoRA adapter model and tokenizer already found in /content/exp/mpnet_lora8_8bit_robust/.

Skipping fine-tuning. To re-train, clear or use a different --output_dir.

In [ ]:
!zip -r /content/file.zip /content/exp

updating: content/exp/ (stored 0%)
updating: content/exp/mpnet_lora8_8bit_robust/ (stored 0%)
updating: content/exp/mpnet_lora8_8bit_robust/training_args.bin (deflated 51%)
updating: content/exp/mpnet_lora8_8bit_robust/trainer_state.json (deflated 75%)
updating: content/exp/mpnet_lora8_8bit_robust/vocab.txt (deflated 53%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/ (stored 0%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/training_args.bin (deflated 51%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/trainer_state.json (deflated 75%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/adapter_model.safetensors (deflated 7%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/adapter_config.json (deflated 50%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/optimizer.pt (deflated 18%)
updating: content/exp/mpnet_lora8_8bit_robust/checkpoint-5618/rng_state.pth (deflated 25%)
updating: content/exp/mpnet_lo

# Bench Marking

In [ ]:
# LoRA + 8‑bit Fine‑Tuning for Extreme Multi‑Label Retrieval (Redesigned for Robustness)
# Author: Shivaram Kumar (Systems Researcher)
# Original Time‑frame: Jan – Feb 2025
# Redesign: For error handling and Colab-friendliness
# Hardware target: single RTX 3060 Max‑Q (original), Google Colab T4 GPU
#
# -----------------------------------------------------------------------------
# This script covers the full reproducible pipeline:
#   • parameter‑efficient fine‑tuning of an MPNet retriever with LoRA (r=8)
#   • 8‑bit Adam optimiser via bitsandbytes
#   • recall@100 evaluation for extreme multi‑label retrieval (Amazon‑670K‑like)
#   • timing / throughput profiler
#   • automatic logging of cost–speed trade‑offs
#
# The code is organised as *one* Python file that can be executed in two modes:
#   1.  TRAIN   – fine‑tune and save the LoRA‑adapted 8‑bit model + tokenizer
#   2.  BENCH   – measure GPU memory, wall‑clock throughput and recall@100
#
# Example usage (after placing dataset in data/Amazon670K):
#   $ python lora_8bit_retrieval_robust.py train \
#       --dataset_path data/Amazon670K \
#       --output_dir exp/mpnet_lora8_8bit_robust
#   $ python lora_8bit_retrieval_robust.py bench \
#       --dataset_path data/Amazon670K \
#       --model_dir exp/mpnet_lora8_8bit_robust
# -----------------------------------------------------------------------------

#
# === FOR GOOGLE COLAB: Run this cell first ===
#
# !pip install torch==2.2.* torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.41.*
# !pip install peft==0.11.*
# !pip install bitsandbytes==0.43.*
# !pip install datasets==2.19.*
# !pip install faiss-cpu==1.8.* # or faiss-gpu if you prefer and have VRAM
# !pip install accelerate==0.28.*
# !pip install rich>=13
# !pip install sentence-transformers # For all-mpnet-base-v2, though AutoModel handles it
#
# ========================================
#


if __name__ == "__main__":
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Clear cache at the very beginning


    import sys

    sys.argv = [
        'my_script_name.py',        # Placeholder for script name
        'bench',                    # <--- CHANGE TO 'bench'
        '--dataset_path', '/content/drive/MyDrive/LORA',
        '--model_dir', '/content/exp/mpnet_lora8_8bit_robust/', # Dir where trained LoRA is
        '--base_model_name_for_bench', 'sentence-transformers/all-mpnet-base-v2', # Original base
        '--recall_k_val', '100',    # Example, default is 100
        # Add other bench-specific args if needed, e.g.:
        # '--val_samples', '1000',
        # '--index_samples', '10000'
    ]
    args = parse_args() # This will now parse bench arguments
    # ... the rest of your if/elif block will now execute bench(args)

    args = parse_args()

    console.print(Panel(f"[bold bright_magenta]Executing Command: {args.command}[/]", expand=False, border_style="bright_magenta"))
    console.print(f"Arguments: {vars(args)}")

    if args.command == "train":
        # For training, args.model_name is the base model to start from
        if not args.model_name: # From p_common, should be set unless explicitly None
             console.print("[bold red]Error: --model_name must be specified for training.[/]")
             sys.exit(1)
        train(args)
    elif args.command == "bench":
        # For benchmarking, args.base_model_name_for_bench is the original base
        # and args.model_dir contains the LoRA adapters.
        if not args.base_model_name_for_bench:
            console.print("[bold red]Error: --base_model_name_for_bench must be specified for benchmarking to load adapters correctly.[/]")
            sys.exit(1)
        bench(args)
    else:
        console.print(f"[bold red]Unknown command: {args.command}[/]")
        sys.exit(1)

╭──────────────────────────╮
│ Executing Command: bench │
╰──────────────────────────╯

Arguments: {'command': 'bench', 'dataset_path': '/content/drive/MyDrive/LORA', 'model_name': None, 
'max_seq_length': 128, 'topk_labels_train': None, 'topk_labels_val': None, 'model_dir': 
'/content/exp/mpnet_lora8_8bit_robust/', 'base_model_name_for_bench': 'sentence-transformers/all-mpnet-base-v2', 
'eval_batch_size': 64, 'val_samples': 10000, 'index_samples': 100000, 'recall_k_val': 100}

Loading sentence-transformers/all-mpnet-base-v2 (8‑bit=True)...

Warning: MPNet models have issues with 8-bit quantization. Loading with fp16 instead.

✓ Model sentence-transformers/all-mpnet-base-v2 loaded successfully (quantized: False).

...

Performing search and calculating recall@100...

FAISS search and recall calculation took: 25.49s.

╭──────────────────────────────────────────────╮
│ ✓ Benchmarking Complete. Recall@100: 100.00% │
╰──────────────────────────────────────────────╯

            Benchmark Summary (Recall@100) - /content/exp/mpnet_lora8_8bit_robust/             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric                                            ┃ Value                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ timestamp                                         │ 2025-05-29T23:37:34                     │
│ model_dir                                         │ /content/exp/mpnet_lora8_8bit_robust/   │
│ base_model_name                                   │ sentence-transformers/all-mpnet-base-v2 │
│ dataset_path                                      │ /content/drive/MyDrive/LORA             │
│ index_samples                                     │ 100000                                  │
│ query_samples                                     │ 10000                                   │
│ recall@100                                        │ 100.0000                                │
│ throughput_qps (queries_encoded_and_searched/sec) │ 311.0056                                │
│ corpus_encode_time_s                              │ 72.1236                                 │
│ faiss_index_build_time_s                          │ 0.8639                                  │
│ total_index_build_time_s                          │ 72.9875                                 │
│ query_encode_time_s                               │ 6.6685                                  │
│ search_and_recall_calc_time_s                     │ 25.4853                                 │
│ gpu_peak_mem_gb                                   │ 0.9305                                  │
└───────────────────────────────────────────────────┴─────────────────────────────────────────┘

Summary saved to → /content/exp/mpnet_lora8_8bit_robust/bench_summary_k100.json